<a href="https://colab.research.google.com/github/Kyungmin-Yu/old_datamining/blob/master/%EB%AA%A8%EB%8D%B8%EB%A7%81_%EC%BD%94%EB%93%9C%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [144]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [187]:
import os
import pandas as pd
import numpy as np
os.chdir("/content/drive/My Drive/data")
dat = pd.read_csv("final_data0723_3.csv", encoding='CP949')

In [188]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8758 entries, 0 to 8757
Data columns (total 33 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   y             8686 non-null   float64
 1   date          8758 non-null   object 
 2   month         8758 non-null   int64  
 3   day           8758 non-null   int64  
 4   wday          8758 non-null   object 
 5   hour          8758 non-null   int64  
 6   temp          8758 non-null   float64
 7   rain          8758 non-null   float64
 8   wind_speed    8758 non-null   float64
 9   humidity      8758 non-null   float64
 10  rain_prob     8758 non-null   float64
 11  rain_type     8758 non-null   int64  
 12  skycondition  8758 non-null   int64  
 13  ymd           8758 non-null   object 
 14  daily         8758 non-null   float64
 15  daily_smooth  8758 non-null   float64
 16  pm10_cat      8737 non-null   object 
 17  pm2_5_cat     8737 non-null   object 
 18  altitude      8737 non-null 

In [189]:
dat = dat.set_index(pd.to_datetime(dat['date']))
submit_dat = dat[dat['y'].isnull()]
submit_dat['y'] = 0
dat = dat[dat['y'].notnull()]

In [192]:
train_1 = dat[(dat["ymd"] < "2020-01-24")]
train_2 = dat[(dat["ymd"] < "2020-03-24")]
train_3 = dat[(dat["ymd"] < "2020-05-24")]

val_1 = dat[((dat["ymd"] > "2020-01-23") & (dat["ymd"] <"2020-01-31"))]
val_2 = dat[((dat["ymd"] > "2020-03-23") & (dat["ymd"] <"2020-03-31"))]
val_3 = dat[((dat["ymd"] > "2020-05-23") & (dat["ymd"] <"2020-05-31"))]

test_1 = submit_dat[(submit_dat["ymd"] == "2020-01-31")]
test_2 = submit_dat[(submit_dat["ymd"] == "2020-03-31")]
test_3 = submit_dat[(submit_dat["ymd"] == "2020-05-31")]

submit_train_1 = dat[(dat["ymd"] < "2020-01-31")]
submit_train_2 = dat[(dat["ymd"] < "2020-03-31")]
submit_train_3 = dat[(dat["ymd"] < "2020-05-31")]

catboost

In [149]:
! pip install catboost

In [150]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import KFold

In [194]:
#cat_var = ["month", "hour", "temp", "wind_speed", "humidity","rain_prob", "skycondition", "rain_type"]
cat_var = ['month', 'hour', 'wind_speed', 'humidity', 'rain_prob', 'daily_smooth', 'azimuth', 'temp_diff', 'ws_mean']

In [195]:
cb_model = CatBoostRegressor(iterations=1000,
                          loss_function='RMSE',
                          eval_metric='RMSE')

In [199]:
## 함수 정의 
def RMSE(y_true,y_pred):
    rmse = (mse(y_true,y_pred))**0.5
    return rmse

def cat_pred (train ,test):
  train_x = train.loc[:, cat_var]
  train_y = train.loc[:,['y']]
  test_x =  test.loc[:, cat_var]
  test_y = test.loc[:,['y']]
  cat_features = list(train_x.select_dtypes(include=['object', 'int64']))
  cb_model.fit(train_x, train_y, 
             eval_set=(test_x,test_y), 
             cat_features=cat_features,
             use_best_model=True,
             verbose=False)
  pred_y = cb_model.predict(test_x)
  true_y = test_y.to_numpy().flatten()
  E = lossloss(true_y, pred_y)
 
  return pred_y

def lossloss(y, y_hat):
  result = []
  for i in range(int(y.shape[0]/24)):
    start = 24*i
    end = (24*(i+1))
    tmp = sum(abs(y[start:end] - y_hat[start:end])*y[start:end]/sum(y[start:end]))/133
    result.append(tmp)
  loss = np.mean(result)
  return loss

def cat_fold_prediction(train, test, N_fold=5):
    kf = KFold(n_splits=N_fold, shuffle=True, random_state=0)
    y_pred_mean = np.zeros(len(test))
    for i, (train_index,val_index) in enumerate(kf.split(train)):
        train_cv = train.iloc[train_index,:]
        val_cv = train.iloc[val_index,:]
        y_pred = cat_pred(train_cv, test)
        y_pred_mean += np.array(y_pred)/N_fold
    
    rmse = RMSE(test['y'],y_pred_mean)
    E = lossloss(test['y'],y_pred_mean)
    print("가중평균: ", E)
    print("RMSE:", rmse) 
    return (rmse, E, y_pred_mean)

In [197]:
pd.isnull(test_1.iloc[0,:]['y'] ) ==True

False

In [154]:
rmse1, E1, cat_pred_1 = cat_fold_prediction(train_1, val_1)
rmse2, E2, cat_pred_2 = cat_fold_prediction(train_2, val_2)
rmse3, E3, cat_pred_3 = cat_fold_prediction(train_3, val_3)

가중평균:  0.07402790191546714
RMSE: 6.302861954044814
가중평균:  0.09030564365143903
RMSE: 9.955758222104135
가중평균:  0.06979896762276575
RMSE: 7.578203128804214


In [155]:
(E1+E2+E3)/3

0.07804417106322396

In [200]:
_, _, cat_0131 = cat_fold_prediction(submit_train_1, test_1)
_, _, cat_0331 = cat_fold_prediction(submit_train_2, test_2)
_, _, cat_0531 = cat_fold_prediction(submit_train_3, test_3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide


가중평균:  nan
RMSE: 14.189389611678688


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide


가중평균:  nan
RMSE: 16.897380529832084


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide


가중평균:  nan
RMSE: 18.316195712282017


SVR

In [157]:
! pip install category_encoders

In [201]:
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
pd.options.mode.chained_assignment = None

In [202]:
svr_var = ['month', 'hour', 'temp', 'rain', 'humidity', 'skycondition', 'daily', 'daily_smooth', 'altitude', 'azimuth', 'sin_time', 'cos_time', 'temp_diff', 'ws_diff',
 'temp_max', 'hum_max', 'ws_max', 'temp_mean', 'hum_mean', 'ws_mean']

In [219]:
## 함수 정의
def mean_encoder(train,test,categorical_var):
    ## mean encoding
    cate_var = categorical_var
    for i in cate_var:
        tg_encoder = ce.target_encoder.TargetEncoder(cols=[i])
        tg_encoder.fit(X = train[i],y = train['y'])
        train[i] = tg_encoder.transform(train[i])
        test[i] = tg_encoder.transform(test[i])
    return (train,test)
# train_1,val_1 = mean_encoder(train_1,val_1,['month','hour','pm10_cat','pm2_5_cat','rain_yn'])

def lossloss(y_true, y_pred):
  ## 가중평균
    result = []
    for i in range(int(y_true.shape[0]/24)):
        start = 24*i
        end = (24*(i+1))
        tmp = sum(abs(y_true[start:end] - y_pred[start:end])*y_true[start:end])/sum(y_true[start:end])/133
        result.append(tmp)
    loss = np.mean(result)
    return loss

def svr_predict(train,test):
    train,val = train,test
    categorical_var = ['month','hour','rain_yn','pm10_cat','pm2_5_cat']
    categorical_var = [v for v in categorical_var if v in train.columns.to_list()]
    train,val = mean_encoder(train,val,categorical_var)
    train_X = train.loc[:, svr_var]
    train_y = train.loc[:,['y']]
    sc_X = StandardScaler()
    sc_y = StandardScaler()
    train_X = sc_X.fit_transform(train_X)
    train_y = sc_y.fit_transform(train_y).reshape(-1)
    val_X = sc_X.transform(val.loc[:, svr_var])
    val_y = sc_y.transform(val.loc[:,['y']])
    regressor = SVR(kernel='rbf')
    regressor.fit(train_X,train_y)
    y_pred = regressor.predict(val_X)
    y_pred = sc_y.inverse_transform(y_pred)
    if sum(val['y'])==0:
      result = (0,0,y_pred)
    else:
      rmse = RMSE(val['y'],y_pred)
      E = lossloss(val['y'],y_pred)
      print('rmse:',rmse,'가중평균:',E)
      result = (rmse, E, y_pred)
    return result

In [218]:
sum(val_1['y'])

2068.448999

In [220]:
rmse1, E1, svr_pred_1 = svr_predict(train_1, val_1)
rmse2, E2, svr_pred_2 = svr_predict(train_2, val_2)
rmse3, E3, svr_pred_3 = svr_predict(train_3, val_3)

rmse: 6.825112376587067 가중평균: 0.07952281158770773
rmse: 9.276474054256466 가중평균: 0.08975456534819355
rmse: 7.216267117348259 가중평균: 0.06052388003012432


In [221]:
(E1+E2+E3)/3

0.0766004189886752

In [222]:
_, _, svr_0131 = svr_predict(submit_train_1, test_1)
_, _, svr_0331 = svr_predict(submit_train_2, test_2)
_, _, svr_0531 = svr_predict(submit_train_3, test_3)

FFNN

In [163]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as mse
import category_encoders as ce
import sys
from keras.layers.advanced_activations import LeakyReLU, PReLU
np.set_printoptions(threshold=sys.maxsize)
np.random.seed(0)

In [164]:
drop_columns = ["date", "wday", "ymd"]

In [165]:
def Scaler(dat_ffnn):
  dat_ffnn["temp"] /= 100
  dat_ffnn["rain"] /= 100
  dat_ffnn["wind_speed"] /= 100
  dat_ffnn["humidity"] /=100
  dat_ffnn["rain_prob"] /=100
  dat_ffnn["altitude"] /=10
  dat_ffnn["azimuth"] /= 100 
  dat_ffnn["temp_diff"] /= 100
  dat_ffnn["ws_diff"] /=100
  dat_ffnn["temp_max"] /= 100
  dat_ffnn["hum_max"] /=100
  dat_ffnn["ws_max"] /=100
  dat_ffnn["temp_mean"] /=100
  dat_ffnn["hum_mean"] /= 100
  dat_ffnn["ws_mean"] /= 100
  result = dat_ffnn.copy()
  return result

In [166]:
def mean_encoder(train,test,categorical_var):
    ## mean encoding
    train = Scaler(train)
    test = Scaler(test)
    cate_var = categorical_var
    train = train.drop(drop_columns,axis=1)
    test = test.drop(drop_columns,axis=1)
    for i in cate_var:
        tg_encoder = ce.target_encoder.TargetEncoder(cols=[i])
        tg_encoder.fit(X = train[i],y = train['y'])
        train[i] = tg_encoder.transform(train[i])
        test[i] = tg_encoder.transform(test[i])
        X_train,y_train = np.array(train.drop(['y'],axis=1)), np.array(train.loc[:,['y']])
        X_valid,y_valid = np.array(test.drop(['y'],axis=1)), np.array(test.loc[:,['y']])
    return (X_train,y_train, X_valid, y_valid)

def lossloss(y, y_hat):
  result = []
  for i in range(int(y.shape[0]/24)):
    start = 24*i
    end = (24*(i+1))
    tmp = sum(abs(y[start:end] - y_hat[start:end])*y[start:end]/sum(y[start:end]))/133
    result.append(tmp)
  loss = np.mean(result)
  return loss

def RMSE(y_true,y_pred):
    rmse = (mse(y_true,y_pred))**0.5
    return rmse

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

def modeling(data_set, model, batchsize = 64, learning_rate = 300):
  weighted = []
  rmse = []
  y_hat_result = []
  for i in range(len(data_set)):
    X_train = data_set[i][0]
    Y_train = data_set[i][1]
    X_val = data_set[i][2]
    Y_val = data_set[i][3]

    model.fit(X_train, Y_train, epochs = 300, batch_size = batchsize, validation_split = 0.3, verbose = 0, callbacks = [mc, early_stopping])
    
    y_hat = model.predict(X_val)
    y_hat = y_hat.flatten()
    y_hat_tmp = y_hat

    tmp_weighted = lossloss(Y_val, y_hat)
    tmp_rmse = RMSE(Y_val, y_hat)

    weighted.append(tmp_weighted)
    rmse.append(tmp_rmse)
    y_hat_result = y_hat_tmp
    print(i, "번째 validiaion 결과는 다음과 같습니다.", ":::", "가중평균: ", tmp_weighted, "RMSE: ", tmp_rmse )

  print("   ")
  print(np.mean(weighted))
 # return y_hat_result
  return weighted, y_hat_result

In [167]:
cate_list = ["month", "skycondition", "pm10_cat", "pm2_5_cat", "rain_yn", "hour" ] #인코딩을 위한 변수

In [168]:
x_train_1, y_train_1, x_val_1, y_val_1 = mean_encoder(train_1, val_1, cate_list)
x_train_2, y_train_2, x_val_2, y_val_2 = mean_encoder(train_2, val_2, cate_list)
x_train_3, y_train_3, x_val_3, y_val_3 = mean_encoder(train_3, val_3, cate_list)

In [169]:
data_set_list = [[x_train_1, y_train_1, x_val_1, y_val_1],
                 [x_train_2, y_train_2, x_val_2, y_val_2],
                 [x_train_3, y_train_3, x_val_3, y_val_3]]

In [170]:
dim_x_train_1 = x_train_1.shape

# 1. 모델 구성하기
model = Sequential()
model.__init__
model.add(Dense(40, input_dim= dim_x_train_1[1], activation='sigmoid'))
model.add(Dense(20))
model.add(Dense(1, activation = "linear"))
# 2. 모델 학습과정 설정하기
model.compile(optimizer = "adamax", loss = root_mean_squared_error, 
              metrics =["mse"])

# 3. 모델 학습시키기
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

In [229]:
def modeling_final(X_train, Y_train, X_test):
  dim_train = X_train.shape

  model_final = Sequential()
  model_final.__init__
  model_final.add(Dense(40, input_dim= dim_train, activation='sigmoid'))
  model_final.add(Dense(20))
  model_final.add(Dense(1, activation = "linear"))
  # 2. 모델 학습과정 설정하기
  model.compile(optimizer = "adamax", loss = root_mean_squared_error, 
                metrics =["mse"])

  # 3. 모델 학습시키기
  early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
  mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)\

  model_final.fit(X_train, Y_train, epochs = 300, batch_size = 64, validation_split = 0.2, verbose = 0, callbacks = [mc, early_stopping])
  y_hat = model_final.predict(X_test)
  y_hat = y_hat.flatten()
 
  return y_hat

In [171]:
E, pred = modeling(data_set_list, model)

0 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.2592723622770918 RMSE:  7.842425348374033
Epoch 00210: early stopping
1 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.39317000681393904 RMSE:  10.910240013017546
Epoch 00190: early stopping
2 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.36322566629061603 RMSE:  9.177025057401611
   
0.33855601179388223


In [231]:
def mean_encoder_final(train,test,categorical_var):
    ## mean encoding
    train = Scaler(train)
    test = Scaler(test)
    cate_var = categorical_var
    train = train.drop(drop_columns,axis=1)
    test = test.drop(drop_columns,axis=1)
    for i in cate_var:
        tg_encoder = ce.target_encoder.TargetEncoder(cols=[i])
        tg_encoder.fit(X = train[i],y = train['y'])
        train[i] = tg_encoder.transform(train[i])
        test[i] = tg_encoder.transform(test[i])
        X_train,y_train = np.array(train.drop(['y'],axis=1)), np.array(train.loc[:,['y']])
        X_valid = np.array(test)
    return (X_train,y_train, X_valid)

In [ ]:
train = Scaler(train)
test = Scaler(test)
cate_var = categorical_var
train = train.drop(drop_columns,axis=1)
test = test.drop(drop_columns,axis=1)
for i in cate_var:
  tg_encoder = ce.target_encoder.TargetEncoder(cols=[i])
  tg_encoder.fit(X = train[i],y = train['y'])
  train[i] = tg_encoder.transform(train[i])
  test[i] = tg_encoder.transform(test[i])
  X_train,y_train = np.array(train.drop(['y'],axis=1)), np.array(train.loc[:,['y']])
  X_valid = np.array(test)
return (X_train,y_train, X_valid)

In [233]:
x_submit_train_1, y_submit_train_1, x_test_1, y_test_1 = mean_encoder(submit_train_1, test_1, cate_list)

ValueError: ignored

In [232]:
x_submit_train_1, y_submit_train_1, x_test_1, y_test_1 = mean_encoder(submit_train_1, test_1, cate_list)
x_submit_train_2, y_submit_train_2, x_test_2, y_test_2 = mean_encoder(submit_train_2, val_2, cate_list)
x_submit_train_3, y_submit_train_3, x_test_3, y_test_3 = mean_encoder(submit_train_3, val_3, cate_list)

ValueError: ignored

In [226]:
ffnn_0131 = modeling_final(x_submit_train_1, y_submit_train_1, x_test_1)
ffnn_0331 = modeling_final(x_submit_train_1, y_submit_train_1, x_test_1)
ffnn_0531 = modeling_final(x_submit_train_1, y_submit_train_1, x_test_1)

TypeError: ignored